In [ ]:
'''
Pseudocode (Manim Math Educational Video Generator)
-. User input prompt to generate certain video based from their needs di gradio
-. Prompt user kemudian diteruskan ke LLM -- Claude
-. Dapat respons berupa code dari claude
-. respons code kemudian masuk ke agent untuk review code (claude)
-. Execute terminal commands
-. Pada gradio muncul video yang telah di generate
'''

In [ ]:
# Imports 
import gradio as gr
from dotenv import load_dotenv
import anthropic
import os
import subprocess
import os
import tempfile
import re
import time
from pathlib import Path

In [ ]:
load_dotenv(override=True)

# Anthropic API Key
anthropic_api_key  = os.getenv('ANTHROPIC_API_KEY')


if anthropic_api_key:
    print(f"Anthropic API key exists and begins {anthropic_api_key[:8]}")

claude         = anthropic.Anthropic()
model          = "claude-3-5-haiku-latest"
reviewer_model = "claude-opus-4-20250514"

In [ ]:
# System prompt code builder
code_builder_system_prompt ="""
You are a manim video expert with exceptional Python coding skills using the manim library exclusively.

You will receive requests for video animations and must create Python code that produces animations centered on the screen at all times.

CENTERING REQUIREMENTS:
- All visual elements (text, shapes, graphs, equations) must be positioned at the center of the screen
- Use ORIGIN, UP, DOWN, LEFT, RIGHT for positioning relative to center
- Apply .move_to(ORIGIN) or .shift() methods to ensure proper centering
- For complex scenes, group elements and center the entire group
- Ensure animations flow from center outward or maintain central focus throughout

SIZING AND SCALING REQUIREMENTS:
- Scale all objects appropriately for laptop screen dimensions (1920x1080 or 1366x768)
- Text should use font sizes between 20-48 for readability
- Shapes and figures must fit within screen bounds with comfortable margins
- Use .scale() method to adjust oversized elements
- Graphs and plots should have appropriate axis ranges and tick spacing
- Mathematical expressions should be legible without being oversized

LAYOUT AND SPACING REQUIREMENTS:
- Prevent any visualization overlap by using proper spacing
- Use strategic positioning with UP*2, DOWN*2, LEFT*3, RIGHT*3 for separation
- Sequence animations temporally to avoid simultaneous conflicting elements
- Use FadeOut previous elements before introducing new ones when necessary
- Maintain clear visual hierarchy and readable spacing between components

PERFORMANCE OPTIMIZATION:
- Minimize object creation and memory usage
- Use efficient animation methods (Transform, FadeIn, FadeOut)
- Consolidate animations using AnimationGroup when possible
- Optimize timing with appropriate run_time parameters
- Remove objects from scene when no longer needed using self.remove()

EXAMPLES OF REQUESTS:
1. Help me make animations to visualize a math case using linear regression
2. Generate me animations that breakdown the process of getting the formula of square area

OUTPUT RULES:
- Respond ONLY in Python code
- No explanatory text, comments, or words outside Python syntax
- No code explanations before or after the code block
- Code must be directly executable with manim
- All animations must maintain center-screen positioning throughout the entire sequence
"""

In [ ]:
# System prompt code peer review
peer_review_system_prompt="""
You are an expert manim code reviewer with exceptional Python optimization skills.

REVIEW TASKS:
1. Perform comprehensive code quality analysis
2. Optimize for maximum performance, efficiency, and execution speed
3. Apply manim best practices and performance patterns
4. Minimize redundant operations and memory usage
5. Optimize animation timing and rendering efficiency

OPTIMIZATION FOCUS AREAS:
- Remove unnecessary object creations and duplications
- Consolidate similar animations using AnimationGroup or Succession
- Use efficient manim methods (Transform vs ReplacementTransform vs FadeTransform)
- Minimize Scene.wait() calls and optimize timing
- Apply proper object lifecycle management (add/remove at optimal times)
- Use vectorized operations where possible
- Optimize positioning and transformation calculations
- Reduce total render time while maintaining visual quality

OUTPUT REQUIREMENTS:
- Respond ONLY in optimized Python code
- No comments, explanations, or non-Python text
- Code must be directly executable with manim
- Must maintain or improve the original animation's visual output
- Focus on measurable performance improvements
"""

In [ ]:
def execute_code(code,output_name="generated_video", quality="medium"):
    
    # Quality mapping
    quality_flags = {
        "low": "-ql",
        "medium": "-qm", 
        "high": "-qh"
    }

    temp_dir = tempfile.mkdtemp()
    temp_file = os.path.join(temp_dir, f"{output_name}_scene.py")

    try:
        
        with open(temp_file, 'w', encoding='utf-8') as f:
            f.write(code)
        
        scene_name = extract_scene_name(code)
        if not scene_name:
            return {
                "success": False,
                "error": "No Scene class found in the code",
                "video_path": None
            }
        
        quality_flag = quality_flags.get(quality, "-qm")
        cmd = [
            "manim",
            temp_file,
            scene_name,
            quality_flag,
            "--output_file", f"{output_name}.mp4",
            "-v", "WARNING"  # Reduce verbosity
        ]
        
        print(f"🎬 Executing: {' '.join(cmd)}")
        
        start_time = time.time()
        result = subprocess.run(
            cmd, 
            capture_output=True, 
            text=True, 
            cwd=temp_dir,  # Run in temp directory
            timeout=300  # 5 minute timeout
        )
        
        execution_time = time.time() - start_time
        print(f"Execution time: {execution_time:.2f} seconds")
        
        if result.returncode == 0:
            # Find the generated video file
            video_path = find_generated_video(temp_dir, output_name, scene_name)
            
            if video_path and os.path.exists(video_path):
                # Move video to current directory
                final_path = f"{output_name}.mp4"
                if os.path.exists(final_path):
                    os.remove(final_path)
                
                os.rename(video_path, final_path)
                
                return {
                    "success": True,
                    "video_path": final_path,
                    "execution_time": execution_time,
                    "scene_name": scene_name
                }
            else:
                return {
                    "success": False,
                    "error": "Video file was not generated",
                    "stdout": result.stdout,
                    "stderr": result.stderr
                }
        else:
            return {
                "success": False,
                "error": "Manim execution failed",
                "stdout": result.stdout,
                "stderr": result.stderr,
                "return_code": result.returncode
            }
    
    except subprocess.TimeoutExpired:
        return {
            "success": False,
            "error": "Execution timed out (5 minutes limit)"
        }
    
    except Exception as e:
        return {
            "success": False,
            "error": f"Unexpected error: {str(e)}"
        }
    
    finally:
        cleanup_temp_files(temp_dir)



In [ ]:
def find_generated_video(base_dir, output_name, scene_name):
    media_dir = os.path.join(base_dir, "media")
    
    if not os.path.exists(media_dir):
        return None
    
    # Search for the video file
    for root, dirs, files in os.walk(media_dir):
        for file in files:
            if file.endswith(".mp4") and (output_name in file or scene_name in file):
                return os.path.join(root, file)
    
    return None

In [ ]:
def extract_scene_name(code):
    pattern = r'class\s+(\w+)\s*\(\s*Scene\s*\)'
    matches = re.findall(pattern, code)
    
    if matches:
        return matches[0]  # Return first found scene class
    
    pattern = r'class\s+(\w+)'
    matches = re.findall(pattern, code)
    
    return matches[0] if matches else None

In [ ]:
def find_generated_video(base_dir, output_name, scene_name):
    
    media_dir = os.path.join(base_dir, "media")
    
    if not os.path.exists(media_dir):
        return None
    
    # Search for the video file
    for root, dirs, files in os.walk(media_dir):
        for file in files:
            if file.endswith(".mp4") and (output_name in file or scene_name in file):
                return os.path.join(root, file)
    
    return None

In [ ]:
def cleanup_temp_files(temp_dir):
    try:
        import shutil
        shutil.rmtree(temp_dir)
    except Exception as e:
        print(f"Warning: Could not clean up temp files: {e}")

In [ ]:
def clean_code_blocks(code):

    # Remove leading/trailing whitespace
    cleaned = code.strip()
    
    # Remove ```python at the beginning
    if cleaned.startswith('```python'):
        cleaned = cleaned[9:]  # Remove '```python'
    elif cleaned.startswith('```'):
        cleaned = cleaned[3:]  # Remove '```'
    
    # Remove ``` at the end
    if cleaned.endswith('```'):
        cleaned = cleaned[:-3]  # Remove '```'
    
    # Remove any remaining leading/trailing whitespace
    cleaned = cleaned.strip()
    
    return cleaned

In [ ]:
def generate_video(user_prompt):

    # Building code
    builder = claude.messages.create(
        model = model,
        max_tokens=512,
        temperature=0.7,
        system=code_builder_system_prompt,
        messages=[
            {"role":"user","content":user_prompt},
        ],
    )
    
    # Generated builder code
    builder_code = builder.content[0].text
    print(builder_code)

    #Review process
    reviewer = claude.messages.create(
        model = reviewer_model,
        max_tokens=1024,
        temperature=0.8,
        system = peer_review_system_prompt,
        messages=[
            {"role":"user","content":builder_code},
        ],
    )

    # Generated reviewer code
    reviewer_code   = reviewer.content[0].text
    production_code = clean_code_blocks(reviewer_code)

    result = execute_code(production_code)

    if result["success"]:
        print(f"Video generated successfully: {result['video_path']}")
        print(f"Total time: {result['execution_time']:.2f} seconds")
        return result['video_path']
    else:
        print(f"Video generation failed: {result['error']}")
        if 'stdout' in result:
            print("STDOUT:", result['stdout'])
        if 'stderr' in result:
            print("STDERR:", result['stderr'])
        return None

In [ ]:
# Gradio Apps mockup

with gr.Blocks() as ui:
    with gr.Row():
        gr.Markdown("Simple Manim Video Generator")
    
    with gr.Row():
        text_interface = gr.Textbox(
            placeholder="Enter your text here...",
            lines = 10,
            max_lines = 10,
            label ="Input text"
        )
        video_interface = gr.Video(label="Generated Video")

    with gr.Row():
        button = gr.Button("Generate video",variant="primary")

    def handle_video_generation(user_prompt):
        try:
            video_path = generate_video(user_prompt)
            if video_path and os.path.exists(video_path):
                return video_path
            else:
                return None
        except Exception as e:
            print(f"Error in video generation: {str(e)}")
            return None

    button.click(
        fn=handle_video_generation,
        inputs=text_interface,
        outputs=video_interface
    )

    
ui.launch()